### Setup

In [41]:
import numpy as np
import os
import pdftotext
from PyPDF2 import PdfFileReader
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import spacy
import pandas as pd
from py2neo import Graph
from py2neo.bulk import create_nodes


#### Initialize Graph Database

In [43]:
graph = Graph("http://localhost:7474/", auth=("neo4j", "berjis89"))
graph.run("UNWIND range(1, 3) AS n RETURN n, n * n as n_sq")

n,n_sq
1,1
2,4
3,9


In [10]:
class MyDict(dict):
    def __init__(self,*args,**kwargs):
        super().__init__(*args,**kwargs)
    def __getitem__(self,key):
        return dict.__getitem__(self, key.lower())
    def __setitem__(self,key,value):
        return dict.__setitem__(self,key.lower(), value)


In [32]:
def get_data(folder_name):
    """
    This function assumes that the folder "folder_name" is stored inside the the notebooks folder.
    Params:
    folder_name: string
    Returns: the paths to the data files in the given folder as a list.
    """
    arr = os.listdir(folder_name)
    return [os.path.join(os.getcwd(),folder_name, a) for a in arr]


def pdftotext_wrapper(input_file, options=None, output_file=None):
    """
    This function wraps the pdftotext command line tool.
    Params:
    input_file: string of path to the input pdf file.
    output_file: string of path to the output text file.
    options: string
    Returns: the text as a string.
    """
    if options is None:
        options = ""

    if output_file is None:
        output_file = ""

    check = os.popen("pdftotext " + options + " " + input_file + " " + output_file).read()
    if check == "":
        return "Success"


def extract_text(path, method):
    """
    This function extracts the text from a pdf file.
    Params:
    path: string
    method: string
    Returns: the text as a string.
    """
    if method == "pdftotext_cli":
        file_name = path.replace(os.path.dirname(data[0])+"/", "").replace(".pdf", "")
        output_dir = os.path.join(os.getcwd(), "texts")
        output_file = os.path.join(output_dir, file_name + ".txt")
        pdftotext_wrapper(data[0], "-raw", output_file) 
        with open(output_file, 'r') as f:
            #return f.read()
            return f.read().replace("\n", " ")
            #return f.readlines()

    if method == "pdftotext_python":
        with open(path, "rb") as f:
            return pdftotext.PDF(f)

    if method == "pypdf2":
        text = []
        with open(path, "rb") as f:
            pdf = PdfFileReader(f)
            text = [pdf.getPage(i).extractText() for i in range(pdf.numPages)]
            return text

def extract_entities(quote):
    words = word_tokenize(quote)
    tags = nltk.pos_tag(words)
    tree = nltk.ne_chunk(tags, binary=False)
    return set(
        " ".join(i[0] for i in t)
        for t in tree if hasattr(t, "label") and t.label() != "NE"
    )

def extract_info(folder, source="Tex"):
    """
    This function extracts the information from a paper using different methods and returns it as a dictionary with the following keys:
    {
        "Author/Authors": string,
        "Title": string,
        "Year": string,
        "Journal": string,
        "Volume": string,
        "Pages": string,
        "Abstract": string,
        "Sections": list of strings,
        "References_Sections": list of pairs of strings (refrence, section),
        "refrences": list of strings,
        "Keywords": string,
        "Language": string,
        "Source": string,
        }
    Params:
    paper: string
    source: string
    Returns: a dictionary with the extracted information.

    Reg Tips:
    1. r"\\author.*?\\\\" -> mathc from \authors command till the first \\ using lazy match in the Tex file.


    """
    info = MyDict()

    if source != "Tex":
        return info
    

    contents = get_data(folder)
 
    tex_files = [ct for ct in contents if os.path.splitext(ct)[1] == '.tex']
    bib_files = [ct for ct in contents if os.path.splitext(ct)[1] == '.bib']


    text = ""
    for tex_file in tex_files:
        with open(tex_file, "r") as f:
            temp = f.read()
            text+= "\n"+temp
    
    refs = ""
    for bib_file in bib_files:
        with open(bib_file, "r") as f:
            temp = f.read()
            refs+= "\n"+temp

    # Remove all comments:
    #text = re.sub(r"\%.*?\n", "", text,  re.DOTALL)



    # Extracting the author(s)
    #authors = re.findall(r"\\author\{(.*?)\}", text)
 
    Author_main = re.findall(r"\\author\[(.*?)\]", text)
    temp_authors = re.findall(r"\\author.*?\\\\", text, re.DOTALL)
    temp_authors = re.findall(r"\].*?\\\\", temp_authors[0], re.DOTALL)
    temp_authors = re.findall(r"(([A-Zéúßäüö]\.?\s?)*([A-Zéúßäüö][a-zéúßäüö]+\.?\s?)+([A-Zéúßäüö]\.?\s?[a-zéúßäüö]*)*)", temp_authors[0], re.DOTALL)
    Authors = [aut[0] for aut in temp_authors]


    info["Author_main"] = "; ".join(Author_main)
    info["Authors"] = "; ".join(Authors)

    # Extracting the title
    title_temp = re.findall(r"\\title.*?]", text, re.DOTALL)
    title = re.findall(r"\[(.*?)]", title_temp[0], re.DOTALL)
    info["Title"] = title[0]

    # Extracting the Abstract
    abstract_temp = re.findall(r"\\begin{abstract}(.*?)\\end{abstract}", text, re.DOTALL)
    
    # remove comments
    abstract = re.sub(r"\%.*?\n", "", abstract_temp[0],  re.DOTALL)
    info["Abstract"] = abstract


    # Find titles of sections
    section_titles = re.findall(r"\\section{(.*?)}", text, re.DOTALL)

    

    # Extract text of each section:

    sections_text = []
    Sections = MyDict()
    for s_t in section_titles:
        section_grammer= r"\\section{" + s_t + "}" + r"(.*?)" + r"\\section"
        temp =  re.findall(section_grammer, text, re.DOTALL)
        sections_text.append(temp)
        Sections[s_t] = MyDict({"text": temp[0]})
    info["Sections"] = Sections


    # Extracting the citeations in each section:
    for s_t in info["Sections"]:
        # Citation style: \cite[][]{ref1, ref2, ref3}     
        temp = re.findall(r"\\citep.*?{(.*?)}", info["Sections"][s_t]["text"], re.DOTALL)
        info["Sections"][s_t]["citations"] = [t.split(",") for t in temp]

        # Citation style: \citet[][]{ref1, ref2, ref3} 
        temp = re.findall(r"\\citet.*?{(.*?)}", info["Sections"][s_t]["text"], re.DOTALL)
        info["Sections"][s_t]["citations"].append([t.split(",") for t in temp])

        # Citation style: \citealp[][]{ref1, ref2, ref3} 
        temp = (re.findall(r"\\citealp.*?{(.*?)}", info["Sections"][s_t]["text"], re.DOTALL))
        info["Sections"][s_t]["citations"].append([t.split(",") for t in temp])



    citations_info = MyDict()
    # Adding citations from bib files to the sections:
    bib_cts = []
    all_bib_cts = []
    for s_t in info["Sections"]:
        all_cts = info["Sections"][s_t]["citations"]
        for cts in all_cts:
            for ct in cts:
                if len(ct)==1:
                    tmp = re.findall(r"(\@\w+.?(?="+ct[0]+r")(.*?))\@", refs, re.DOTALL)
                else:
                    tmp = re.findall(r"(\@\w+.?(?="+ct+r")(.*?))\@", refs, re.DOTALL)
              #  print(tmp)
                bib_cts.append(tmp)
            all_bib_cts.append(bib_cts)
        citations_info[s_t] = all_bib_cts

            

    
    return info, citations_info

In [33]:
papers = get_data("Texs")

In [34]:
a0,b0 = extract_info(folder=papers[0])

In [35]:
a1,b1 = extract_info(folder=papers[3])

In [36]:
data = [a0,a1]

In [55]:
a0["sections"]["introduction"]["citations"]

[['Espaillat14'],
 ['Andrews11'],
 ['Pietu05'],
 ['Hughes07'],
 ['Casassus13', 'Avenhaus14'],
 ['Espaillat14'],
 ['Carmona14', 'Bruderer14'],
 ['vanderMarel2015'],
 ['Birnstiel2012'],
 ['AlexanderPP6'],
 ['-9'],
 ['Rosenfeld14'],
 ['Rosenfeld14'],
 ['crida06'],
 ['pm04', 'Fouchet07', 'Zhu12'],
 ['Pinilla12', 'JohansenPP6'],
 ['Zhu12', 'Zhu14'],
 ['Zhu11'],
 ['Casassus13', 'Fukagawa14'],
 ['vanderMarel13'],
 ['Tang12'],
 ['Andrews11all', 'Perez14'],
 ['Isella13'],
 ['Brown09', 'Perez14'],
 ['vanDishoeck2015'],
 ['lovelace99'],
 ['PP84'],
 ['Bae15'],
 ['Lyra09', 'LinMK12'],
 ['VT06', 'Regaly12', 'Faure14', 'Lyra15', 'Flock15'],
 ['ZhuStone14'],
 ['Lesur14', 'Bai15', 'Gressel15'],
 [],
 []]

In [44]:
create_nodes(graph.auto(), data, labels={"Paper"})

ClientError: [Statement.TypeError] Property values can only be of primitive types or arrays thereof. Encountered: Map{concluding remarks -> Map{text -> String("
% ===============
\label{sec:conclusion}
We examine in this paper the concentration properties of large dust
grains in protoplanetary transition discs characterised by a lopsided
crescent-shaped distribution in the gas surface density. 2D
hydrodynamical simulations have been carried out including both gas
and dust, with dust modelled as test particles undergoing gas drag
(Epstein regime). The lopsided distribution in the gas is obtained by
adopting a ring-like gas density profile that becomes unstable against
the Rossby-wave instability.  The instability saturates into a
large-scale vortex that efficiently traps the grains. This trapping
mechanism is often invoked to interpret the crescent-shaped continuum
emission imaged by sub-millimetre interferometers in some transition
discs \citep[e.g.,][]{Regaly12}.

In accompanying Pape

In [ ]:
a["Sections"].keys()

In [ ]:
b["Introduction"]

In [ ]:
a["Sections"]['RESULTS OF HYDRODYNAMICAL SIMULATIONS'.lower()]["citations"]

In [ ]:
a_df = pd.DataFrame(a["Sections"])

In [ ]:
a_df.head()

In [ ]:
b

In [ ]:

paper = get_data(papers[2])
paper


In [ ]:
a["Sections"]["Introduction"]

In [ ]:
sent = sent_tokenize(t) 
print(sent)

In [ ]:
#sent = nltk.corpus.treebank.tagged_sents()[22]
words = word_tokenize(t)
words_tagged = nltk.pos_tag(words)

#print(nltk.ne_chunk(words_tagged, binary=False))
tree = nltk.ne_chunk(words_tagged, binary=False)
print(tree)


In [ ]:
IN = re.compile(r'.*\bin\b(?!\b.+ing)')
#for doc in nltk.corpus.ieer.parsed_docs('NYT_19980315'):

#VAN = re.compile(words_tagged, re.VERBOSE)
for doc in sent_tagged:
    for rel in nltk.sem.extract_rels('ORG', 'LOC', doc, corpus='ieer', pattern = IN):
        print(nltk.sem.rtuple(rel))

In [ ]:
nltk.download('ieer')

In [ ]:
nlp = spacy.load("en_core_web_sm")
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")
for token in doc:
    print(token.text, token.pos_, token.dep_)

### Pipline:

In [ ]:
data = get_data("pdfs")

In [ ]:
data[0]

In [ ]:
t = extract_text(data[1], "pdftotext_cli")

In [ ]:
print(t)

In [ ]:
extract_entities(t)

In [ ]:
tt = sent_tokenize(t, language='english', preserve_line=False)

In [ ]:
words = word_tokenize(t[10])
tags = nltk.pos_tag(words)
tree = nltk.ne_chunk(tags, binary=True)

In [ ]:
tree.draw()

In [ ]:
print(nltk.ne_chunk(tt[10], binary=True))